In [1]:
from bs4 import BeautifulSoup
import urllib 
import requests
import os
import pandas as pd
import cv2
from PIL import Image, ImageSequence
import shutil
from urllib.request import Request, urlopen

def crowling_web(url):
    fp = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    source = urlopen(fp).read();
    soup = BeautifulSoup(source, 'html.parser')
    
    return soup

def find_imglink(link):
    soup = crowling_web(link)
    soup_brand_li= soup.find("div", class_='section_product_list').findAll("div", class_='list_img')

    li_brand_img_link = list()
    for img_block in soup_brand_li:
        img_link = img_block.find('a')['href']
        li_brand_img_link.append('https:'+img_link)
        
    return li_brand_img_link

def extract_cate(soup):
    soup3 = crowling_web(soup)
    li_cate = soup3.find('p', class_='item_categories').findAll('a')
    if len(li_cate)==3:
        cate_big=li_cate[0].get_text()
        cate_small=li_cate[1].get_text()
        company = li_cate[2].get_text().replace('\n', '').replace(' ', '').replace('(', '').replace(')', '')
        
        
    elif len(li_cate)==1:
        cate_big = 'None'
        cate_small = 'None'
        company = li_cate[0].get_text().replace('\n', '').replace(' ', '').replace('(', '').replace(')', '')
    
    
    return cate_big, cate_small, company

if __name__ == '__main__':
    url = "https://store.musinsa.com/app/contents/brandshop"
    soup = crowling_web(url)
    
    soup_brand_li= soup.findAll("li", class_='brand_li')
    dict_shoplink = list()
    dict_shopcnt =dict()

    # 브랜드 선정 및 링크 저장
    for i, s in enumerate(soup_brand_li):
        soups = s.findAll('a')
        link=soups[1]['href']
        shop_name = soups[1].get_text().split('(')[0][:-1]
        shop_num = soups[1].get_text().split('(')[1].replace(')', '').replace(',', '')
        dict_shopcnt[shop_name]=int(shop_num)

        if int(shop_num)>800:
            dict_shoplink.append([shop_name, link])
            
    print('company count : ', len(dict_shoplink))        
    category = ['상의', '아우터', '바지', '원피스', '스커트']
    data_li = list()

#     for shop_n in range(len(dict_shoplink)):
    for shop_n in range(50):
        
        
        print('company : {}/{}'.format(shop_n,len(dict_shoplink)))
#         if shop_n==3:
#             break
    #     shop_n=2
        print(dict_shoplink[shop_n][1])

        # page 정보 확인 및 추가
        soup3 = crowling_web(dict_shoplink[shop_n][1])
        page_total = int(soup3.find('span',class_='totalPagingNum').get_text())

        for page_num in range(1, page_total+1):
            print('page_num:{}/{}'.format(page_num, page_total))
            url = dict_shoplink[shop_n][1]+'?category3DepthCodes=&category2DepthCodes=&category1DepthCode=&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods=&timeSale=&includeKeywords=&sortCode=pop_category&tags=&page={}&size=90&listViewType=small&campaignCode='.format(page_num)
    #         print('url: ', url)
            li_brand_img_link = find_imglink(url)


            for j, brank_img_link in enumerate(li_brand_img_link):
                try:
                    cate_big, cate_small, company = extract_cate(brank_img_link)
    #                 print(cate_big)
                    if cate_big in category:

                            if j%10==0:
                                print('processing : {}/{}'.format(j, len(li_brand_img_link)))

                            soup3 = crowling_web(brank_img_link)


                            # 저장폴더 생성
                            imageStr = "./data_800/{}/{}/{}".format(company,cate_big, cate_small)
                            if not os.path.isdir(imageStr):
                                os.makedirs(imageStr)


                            li_img_crowl = soup3.find('ul', class_='product_thumb').findAll('img')
                            name = soup3.find('span', class_='product_title').find('em').get_text()
                            print(name)
                            format_img = soup3.find('div', class_='product-img').find('img')['src']

                            for img_crowl in li_img_crowl:

                                    img = img_crowl['src']
                                    if 'jpg' in img or 'bmp' in img or 'png' in img:
                                        img_name = img.split('/')[-1][:img.split('/')[-1].rindex('_')+1]+'500'+img[-4:]
                                    #     print('img',img)
                                        if 'goods_img' in img:
                                            imgURL = 'https:'+format_img[:format_img.rindex('/')+1]+img_name
                                        else:
                                            imgURL = 'https:'+format_img[:format_img.rindex('/')+1].replace('goods_img', 'prd_img')+img_name

                                        data_li.append([company, img_name, name])
                                        urllib.request.urlretrieve(imgURL, os.path.join(imageStr, img_name))

                except:
                    print('error',dict_shoplink[shop_n][1])
                    
        df = pd.DataFrame(data_li, columns=['company','file_name', 'product_name'])
        df.to_csv('data.csv', encoding='utf-8-sig', index=False)

company count :  155
company : 0/155
https://www.musinsa.com/brands/adidas
page_num:1/8
M 3S 플리스 오픈 팬츠 - 블랙 / GK9267
R.Y.V 스웨트셔츠 - 카키 / GD9275
에센셜 3S 윈드 팬츠 - 블랙:화이트 / DQ3100
아디브레이크 팬츠 - 블랙 / GN2807
릴랙스 오버사이즈 크루넥 - 블랙 / CM1407
로고 트레이닝 팬츠 - 블랙 / H37070
로고 트레이닝 팬츠 - 그레이:실버 / H37071
3S 트랙수트 세트 - 블랙 / GK9651
3 ST 크루 - 블랙 / DV1555
M PG 7/8 조거 - 블랙 / FK1757
로고 스웨트 셔츠 - 블랙 / H37067
processing : 20/90
모던 유로파 트랙팬츠 - 블랙:화이트 / GJ6745
클래식 스웨트 팬츠 - 블랙:화이트 / GD2081
락업 트랙팬츠 - 블랙 / H41387
에센셜 3S T 팬츠 FL - 블랙:화이트 / DQ3093
M 3S 플리스 오픈 팬츠 - 그레이 / GK9270
슈무 후디 - 블랙 / GD3544
로고 스웨트 셔츠 - 라이트 그레이 / H37068
processing : 30/90
아디브레이크 팬츠 - 블랙:화이트 / GJ6580
아웃라인 크루 - 블랙 / FM3856
3-스트라이프 트레이닝 팬츠 - 와인 / CM1410
processing : 40/90
슈무 팬츠 - 블랙 / GE4252
릴랙스 오버사이즈 크루넥 - 베이지 / CM1408
팬츠 3S M - 블랙 / FS6897
유틸리티 윈드브레이커 재킷 - 블랙 / GF4019
파이어버드 트랙팬츠 - 블랙 / GN3517
에센셜 3S 윈드 팬츠 - 네이비:화이트 / DU0453
3스트라이프 우븐 팬츠 - 블랙 / GK8982
M 팩 숏 팬츠 - 블랙 / FL0243
하프 집업 - 블랙 / FS7571
머스트해브 우븐 팬츠 - 블랙 / FM5428
3S 트랙수트 세트  - 네이비 / GK9658
processing : 

In [2]:
print('done')

done
